In [ ]:
import os
import pandas as pd
from IPython.display import display

In [ ]:
# pd.set_option('display.max_rows', 10000)    
# pd.set_option('display.max_columns', 500)  # これらを実行するとDataFrameを出力する際に省略されずにすべて表示されるようになる(重くなる)

## 保存先の設定

user_id: 自身のユーザーID
workspace: jupyterをマウントしたディレクトリ(run_jupyter内の--notebook-dirに指定されているディレクトリ)から、保存先ディレクトリへの相対パス

In [ ]:
user_id = "u01094"                                
workspace = "workspace/mizuho/work/processed_data"

In [ ]:
base_dir = "/home/u00232/dss/share24S/mizuho/work/processed_data" # 変更の必要なし
data_dir = "提供データ"                                           # 提供データ or 運用結果計算例
save_dir = os.path.join("/home", user_id, workspace)              # 保存先ディレクトリ

In [ ]:
files = ["学習用データ.xlsx", "train_indicator_data.csv"] # 処理を行うファイル

In [ ]:
# # ディレクトリ内のすべてのファイルを取得
# files = [file for file in os.listdir(os.path.join(base_dir, data_dir))]

## データ処理用にファイルの読み込み、書き出しを行う関数

引数のprocess_funcを変更することで、自由な処理を行える

In [ ]:
def read_process_save(method, process_func):
    for file in files:
        # splitextは例えば"example.txt"を "example", ".txt"として返す
        base_name, extension = os.path.splitext(file)
        if extension not in [".xlsx", ".csv"]:
            raise ValueError("extension can only be 'xlsx' or 'csv'")

        file_path = os.path.join(base_dir, data_dir, file)

        # ファイルを読み込む
        print(f"reading '{file_path}' ...")
        if extension == ".xlsx":
            df = pd.read_excel(file_path, header=6)
        else:
            df = pd.read_csv(file_path, header=6, sep=';')

        # NaNを削除するなどの処理を行う
        print("processing ...")
        processed_df = process_func(df)

        # 出力ファイル名を設定(csv)
        output_dir = os.path.join(save_dir, method)
        output_file = os.path.join(output_dir, base_name + ".csv")

        # 出力ディレクトリを作成
        os.makedirs(output_dir, exist_ok=True)

        # 整形データを別ファイルに出力
        processed_df.to_csv(output_file, index=False)

        print("saved to " + output_file)
        display(processed_df)

## 処理用の関数

上であげたprocess_func。
DataFrameを受け取って自由に処理を施し、DataFrameを返す。
例として3つ挙げている。

- delete_all_nan_columns: すべてがNaNのカラムをすべて削除する
- delete_all_nan: すべてがNaNのカラムを削除したのち、一つでもNaNの存在するレコードを削除する
- delete_any_nan_columns: 一つでもNaNの存在するカラムをすべて削除する

In [ ]:
def delete_all_nan_columns(df):
        # NaNのみのカラムを削除
        processed_df = df.dropna(axis=1, how='all')
        
        return processed_df

In [ ]:
def delete_all_nan(df):
        # NaNのみのカラムを削除
        df_droped_all_nan_columns = df.dropna(axis=1, how='all')

        # NaNが存在しないレコードを削除
        df_droped_all_nan = df_droped_all_nan_columns.dropna()
        
        return df_droped_all_nan

In [ ]:
def delete_any_nan_columns(df):
    df_droped_any_nan_columns = df.dropna(axis=1)
    
    return df_droped_any_nan_columns

## 実行方法

read_process_save関数に、

1. 保存ディレクトリの名前 (method)
2. 処理用の関数           (process_func)

の二つの引数を渡すと

save_dir内のmethodディレクトリに保存される

In [ ]:
# すべてがNaNであるカラムをすべて削除

read_process_save("delete_all_nan_columns", delete_all_nan_columns)

In [ ]:
# すべてがNaNであるカラムを削除し、NaNが一つでも存在するレコードをすべて削除

read_process_save("delete_all_nan", delete_all_nan)

In [ ]:
# NaNが一つでも存在するカラムをすべて削除

read_process_save("delete_any_nan_columns", delete_any_nan_columns)

## おまけ

レコードを削除してしまう手法を用いると、評価用のデータとずれが生じてしまうため、Datesカラムを参照して一致するデータのみを新たな評価用データとして保存する

train_validation_match は二重配列で、[train_data, validataion_data]の組のリスト

filter_validation_by_train_dates関数には、read_process_save関数に渡したmethodを渡す。

In [ ]:
train_validation_match = [["学習用データ.xlsx", "学習用正解ラベル.xlsx"]]

In [ ]:
def filter_validation_by_train_dates(method):
    for train_file, validation_file in train_validation_match:
        # splitextは例えば"example.txt"を "example", ".txt"として返す
        _, train_extension = os.path.splitext(train_file)
        if train_extension not in [".xlsx", ".csv"]:
            raise ValueError("train extension can only be 'xlsx' or 'csv'")
            
        train_path = os.path.join(save_dir, method, train_file)

        # trainファイルを読み込む
        print(f"reading train file '{train_path}' ...")
        if train_extension == ".xlsx":
            train_df = pd.read_excel(train_path)
        else:
            train_df = pd.read_csv(train_path, sep=';')
        
        validation_base_name, validation_extension = os.path.splitext(validation_file)
        if validation_extension not in [".xlsx", ".csv"]:
            raise ValueError("validation extension can only be 'xlsx' or 'csv'")
            
        validation_path = os.path.join(base_dir, data_dir, validation_file)

        # validationファイルを読み込む
        print(f"reading validation file '{validation_path}' ...")
        if validation_extension == ".xlsx":
            validation_df = pd.read_excel(validation_path)
        else:
            validation_df = pd.read_csv(validation_path, sep=';')
                
        # validationにおいて、trainに残っているDatesに一致する行のみを抽出する
        print("filtering validation data by date ...")
        train_dates = train_df["Dates"]
        filtered_validation_df = validation_df[validation_df["Dates"].isin(train_dates)]
        
        # 出力ファイル名を設定(csv)
        output_dir = os.path.join(save_dir, method)
        output_file = os.path.join(output_dir, validation_base_name + ".csv")

        # 整形データを別ファイルに出力
        filtered_validation_df.to_csv(output_file, index=False)

        print("saved to " + output_file)
        display(filtered_validation_df)

In [ ]:
filter_validation_by_train_dates("delete_all_nan")